In [1]:
import numpy as np
import pandas as pd

In [2]:
train=pd.read_csv("train.csv")

In [3]:
test=pd.read_csv("test.csv")

In [4]:
train.head()

,id,breath_id,R,C,time_step,u_in,u_out,pressure
0,1,1,20,50,0.000000,0.083334,0,5.837492
1,2,1,20,50,0.033652,18.383041,0,5.907794
2,3,1,20,50,0.067514,22.509278,0,7.876254
3,4,1,20,50,0.101542,22.808822,0,11.742872
4,5,1,20,50,0.135756,25.355850,0,12.234987


In [5]:
train.corr()['pressure']

id          -0.002402
breath_id   -0.002394
R            0.015976
C           -0.036727
time_step   -0.524829
u_in         0.308136
u_out       -0.614910
pressure     1.000000
Name: pressure, dtype: float64

In [6]:
train=train.drop(['breath_id'],axis=1)
test=test.drop(['breath_id'],axis=1)

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6036000 entries, 0 to 6035999
Data columns (total 7 columns):
 #   Column     Dtype  
---  ------     -----  
 0   id         int64  
 1   R          int64  
 2   C          int64  
 3   time_step  float64
 4   u_in       float64
 5   u_out      int64  
 6   pressure   float64
dtypes: float64(3), int64(4)
memory usage: 322.4 MB


In [8]:
train=train.drop(['id'],axis=1)
test=test.drop(['id'],axis=1)

In [9]:
X=train.drop('pressure',axis=1)
y=train['pressure']

In [10]:
train.shape

(6036000, 6)

In [11]:
test.shape

(4024000, 5)

In [12]:
from tensorflow import keras 
from tensorflow.keras import layers


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM,GRU,SimpleRNN, RNN, Input, Bidirectional
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from sklearn.model_selection import GroupKFold

from tensorflow.keras.optimizers.schedules import ExponentialDecay

from sklearn.metrics import mean_squared_error as mse

In [14]:

def create_model1():    
    
    model=Sequential()
    model.add(SimpleRNN(10,input_shape= [None,1]))
    model.add(Dense(1))
    model.compile(loss='mae',optimizer='adam')

#   model.compile(loss='mean_squared_error',optimizer='adam')
    return model
    
model1 = create_model1()



In [15]:
scaler = MinMaxScaler()
df = scaler.fit_transform(train)

In [16]:
def create_dataset(dataset, look_back, look_ahead):
    xdat, ydat = [], []
    for i in range(len(df) - look_back -look_ahead):
        xdat.append(df[i:i+ look_back ,0])
        ydat.append(df[i+ look_back : i + look_back + look_ahead,0])
    xdat, ydat = np.array(xdat), np.array(ydat).reshape(-1,look_ahead)
    return xdat, ydat

In [17]:
look_back = 50
look_ahead = 1

xdat, ydat = create_dataset(df, look_back = look_back, look_ahead = look_ahead)

# We only want to forecast a single value for each series => target is a column
print(xdat.shape, ydat.shape)

(6035949, 50) (6035949, 1)


In [18]:
def prepare_split(xdat, ydat, cutoff = 5000, timesteps = 50):
    xtrain, xvalid = xdat[:cutoff,:], xdat[cutoff:,]
    ytrain, yvalid = ydat[:cutoff,:], ydat[cutoff:,]

    # reshape into [batch size, time steps, dimensionality]
    xtrain = xtrain.reshape(-1, timesteps, 1)
    xvalid = xvalid.reshape(-1, timesteps, 1)

    return xtrain, ytrain, xvalid, yvalid 

In [19]:
xtrain, ytrain, xvalid, yvalid = prepare_split(xdat, ydat, cutoff = 5000, timesteps = look_back)

print(xtrain.shape, xvalid.shape, ytrain.shape, yvalid.shape)

(5000, 50, 1) (6030949, 50, 1) (5000, 1) (6030949, 1)


In [20]:
early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 
                           patience = 5, mode = 'min', verbose = 1,
                           restore_best_weights = True)

model1.fit(xtrain, ytrain, validation_data=(xvalid, yvalid), 
                  epochs = 4, batch_size = 16, callbacks=[ early_stop])  

Epoch 1/4
313/313 [==============================] - 2410s 8s/step - loss: 0.0781 - val_loss: 0.0231
Epoch 2/4
313/313 [==============================] - 2270s 7s/step - loss: 0.0276 - val_loss: 0.0200
Epoch 3/4
313/313 [==============================] - 2286s 7s/step - loss: 0.0248 - val_loss: 0.0165
Epoch 4/4
313/313 [==============================] - 2192s 7s/step - loss: 0.0233 - val_loss: 0.0161


In [22]:
y_pred=model1.predict(test)

125750/125750 [==============================] - 331s 3ms/step


In [25]:
submission=pd.read_csv('sample_submission.csv')

In [26]:
submission.head()

,id,pressure
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [27]:
submission['pressure']=y_pred

In [28]:
submission.to_csv('submission.csv')

In [29]:
submission.head()

,id,pressure
0,1,0.836285
1,2,1.421021
2,3,1.484775
3,4,1.493312
4,5,1.494529
